In [ ]:
# parameters
#file = r'C:\Users\roh\Downloads\1602bbe3-3ba6-4d5f-8f80-79cfe019dbc5.xlsx'
#file = r'C:\Users\roh\Downloads\countrystoreproduct.xlsx'

# excel download from DClick
file = 'C:/Users/roh/Downloads/d5c7a4f7-b9a7-4d1e-904e-ce7349e0f27c.xlsx'

# country parameter
country = ['ALL'] # 'Korea'
category_field1 = 2 #0
category_field2 = 4 #1


In [ ]:
# Parameters
file = "C:/Users/roh/Downloads/d5c7a4f7-b9a7-4d1e-904e-ce7349e0f27c.xlsx"
country = "All"


In [ ]:
import pandas as pd
import numpy as np

# sample of excel download from DClick
df1 = pd.read_excel(open(file, 'rb'), sheet_name='Sheet1')
df1

In [ ]:
# tablulating subtotals dynamically in pandas table
# https://towardsdatascience.com/tabulating-subtotals-dynamically-in-python-pandas-pivot-tables-6efadbb79be2

def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
    '''
    Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
    
    Args:
    - df - dataframe used in pivot table
    - values - values used to aggregrate
    - indices - ordered list of indices to aggregrate by
    - columns - columns to aggregrate by
    - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
    - fill_value - value used to in place of empty cells
    
    Returns:
    -flat table with data aggregrated and tabulated
    
    '''
    listOfTable = []
    for indexNumber in range(len(indices)):
        n = indexNumber+1
        table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value).reset_index()
        for column in indices[n:]:
            table[column] = ''
        listOfTable.append(table)
    concatTable = pd.concat(listOfTable).sort_index()
    concatTable = concatTable.set_index(keys=indices)
    return concatTable.sort_index(axis=0,ascending=True)



In [ ]:
# apply pivot table to excel data

df = pd.read_excel(open(file, 'rb'), sheet_name='Sheet1')
if country[0].lower() == 'all':
    new_table = df
else:
    new_table = df[df['Country'].isin(country)]
values = list(df.columns)[4:]  # define list of key facts - from column 5 onwards
indices = list(df.columns)[category_field1:category_field2] # define list of categorisation columns
#columns = list(df.columns)[2:3]
df = pivot_table_w_subtotals(df=new_table, indices=indices, columns=[], values=values, \
          aggfunc='sum',fill_value='')#.sort_values([('Day Net Sales (kEUR) TY')], ascending=False)
#df.sort_values(['Country','YTD Net Sales (kEUR) TY'], ascending=[1,0], inplace=True)
#df.sort_values(['YTD Net Sales (kEUR) TY'], ascending=[0], inplace=True)

#df['equal_or_lower_than_4?'] = df['set_of_numbers'].apply(lambda x: 'True' if x <= 4 else 'False')
#df['Day chg']=df['Day Net Sales (kEUR) LY'].apply(lambda x: df['Day Net Sales (kEUR) TY']/df['Day Net Sales (kEUR) LY']-1
#                                                 if x!=0 else 0)

df['Day chg']=df['Day Net Sales (kEUR) TY']/df['Day Net Sales (kEUR) LY']-1
df['WTD chg']=df['WTD Net Sales (kEUR) TY']/df['WTD Net Sales (kEUR) LY']-1
df['MTD chg']=df['MTD Net Sales (kEUR) TY']/df['MTD Net Sales (kEUR) LY']-1
df['YTD chg']=df['YTD Net Sales (kEUR) TY']/df['YTD Net Sales (kEUR) LY']-1
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df=df[['Day Net Sales (kEUR) TY','Day Net Sales (kEUR) LY','Day chg',
    'WTD Net Sales (kEUR) TY','WTD Net Sales (kEUR) LY','WTD chg',
    'MTD Net Sales (kEUR) TY','MTD Net Sales (kEUR) LY','MTD chg',
    'YTD Net Sales (kEUR) TY','YTD Net Sales (kEUR) LY','YTD chg']]

#df.crosstab()
#df.sort_values(['Retail product L1','YTD Net Sales (kEUR) TY'], ascending=[1,0], inplace=True)
# https://towardsdatascience.com/how-to-do-a-custom-sort-on-pandas-dataframe-ac18e7ea5320
df.sort_values(['Retail product L1','YTD Net Sales (kEUR) TY'], ascending=[0,0], inplace=True)
df

In [ ]:
# apply style sheet / formatting to table

import jinja2  # pip install Jinja2
def color_negative_red(value):
    """
    Colors elements in a dateframe
    green if positive and red if
    negative. Does not color NaN
    values.
    """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    #return 'color: %s' % color
    return f"color:{color};background-color:lightyellow"

def bold(val, props=''):
    return "font-weight:bold;color:black;background-color:darkgrey;text-align:top"

def highlight_max1(s, props=''):
    return np.where(s != 0.0001, props, '')

def highlight_max(s, props=''):
    return [(props if 'Thailand' in s else None) for s in s.index]

def color_red(val):
    return "color: red"

def color_third_red(value):
    return [('color: red' if 'Third' in x else 'color: black') for x in val.index]

def highlight_x(s):
    return ["vertical-align:text-top;" if "" in v else "" for v in s]
#df.style.apply_index(highlight_x, axis="columns", level=[0, 1])



df_style = (df.style
    .set_caption('Daily Sales Report - ' + ', '.join(country))
    .set_table_styles([
    {'selector': 'caption', 'props': [('font-weight', 'bold'),('color', 'darkblue'),('font-size', '16px')]},
    {'selector': 'th', 'props': [('border-style','solid'),('border-color','lightgrey'),('border-width','0.1px')]}
    ])            
    .applymap(color_negative_red, subset=[col for col in df.columns if 'chg' in col])
    .applymap(bold, subset=slice_)
    .apply_index(highlight_x, axis="index", level=[0])
    .format(precision=0, na_rep='na', thousands=",", 
            formatter={'Day chg': "{:.0%}",
                       'MTD chg': "{:.0%}",
                       'YTD chg': "{:.0%}"
                      })    
)


display(df_style)


In [ ]:
# export entire table with style formatting as HTML code for embedding in an email template
print(df_style.to_html())